In [ ]:
import os

spark_version = "spark-3.3.1"
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,067 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,561 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [ ]:
# create df manually
dataframe = spark.createDataFrame([
                                    (0, "Here is our dataframe"),
                                    (1, "We are making one from scratch"),
                                   (2, "This will look similar to Pandas dataframe")],
                                  ["id", "words"])
dataframe.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|Here is our dataf...|
|  1|We are making one...|
|  2|This will look si...|
+---+--------------------+



In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_16/food.csv"
# downloads files across all clusters
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("food.csv"), sep=",", header=True)
#SparkFiles.getRootDirectory() # has the downloaded file
df.show() # show is an action

+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    0|
|  sushi|   12|
|chinese|   10|
+-------+-----+



In [ ]:
#Print the schema
df.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: string (nullable = true)



In [ ]:
#Show columns
df.columns

['food', 'price']

In [ ]:
# Describe data
df.describe()

DataFrame[summary: string, food: string, price: string]

In [ ]:
# Price in the df is a string. Convert it to integer or floating point.
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

# Next we need to create the list of struct fields
schema = [StructField("food", StringType(), True), StructField("price", IntegerType(), True),]
schema

[StructField('food', StringType(), True),
 StructField('price', IntegerType(), True)]

In [ ]:
# Pass in our fields
final = StructType(fields=schema)
final

StructType([StructField('food', StringType(), True), StructField('price', IntegerType(), True)])

In [ ]:
# Read our data with our new schema
dataframe = spark.read.csv(SparkFiles.get("food.csv"), schema=final, sep=",", header=True)
dataframe.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: integer (nullable = true)



In [ ]:
# Transforming data
dataframe['price']

Column<'price'>

In [ ]:
type(dataframe['price'])

pyspark.sql.column.Column

In [ ]:
dataframe.select('price')

DataFrame[price: int]

In [ ]:
type(dataframe.select('price'))

pyspark.sql.dataframe.DataFrame

In [ ]:
dataframe.select('price').show()

+-----+
|price|
+-----+
|    0|
|   12|
|   10|
+-----+



In [ ]:
# Add a new column
# Duplicates the price column with another header
# To store the transformations on df, assign transformation(not action) to same or different df=df.withColumn()
dataframe.withColumn('newprice',dataframe['price']).show()

+-------+-----+--------+
|   food|price|newprice|
+-------+-----+--------+
|  pizza|    0|       0|
|  sushi|   12|      12|
|chinese|   10|      10|
+-------+-----+--------+



In [ ]:
# Update column name
dataframe.withColumnRenamed('price','newerprice').show()

+-------+----------+
|   food|newerprice|
+-------+----------+
|  pizza|         0|
|  sushi|        12|
|chinese|        10|
+-------+----------+



In [ ]:
# Double the price
dataframe.withColumn('doubleprice',dataframe['price']*2).show()

+-------+-----+-----------+
|   food|price|doubleprice|
+-------+-----+-----------+
|  pizza|    0|          0|
|  sushi|   12|         24|
|chinese|   10|         20|
+-------+-----+-----------+



In [ ]:
# Add a dollar to the price
dataframe.withColumn('add_one_dollar',dataframe['price']+1).show()

+-------+-----+--------------+
|   food|price|add_one_dollar|
+-------+-----+--------------+
|  pizza|    0|             1|
|  sushi|   12|            13|
|chinese|   10|            11|
+-------+-----+--------------+



In [ ]:
# Half the price
dataframe.withColumn('half_price',dataframe['price']/2).show()

+-------+-----+----------+
|   food|price|half_price|
+-------+-----+----------+
|  pizza|    0|       0.0|
|  sushi|   12|       6.0|
|chinese|   10|       5.0|
+-------+-----+----------+



In [ ]:
dataframe.select("price").collect()

[Row(price=0), Row(price=12), Row(price=10)]

In [ ]:
dataframe.show()
# assign all transformations to a dataframe to see the changes

+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    0|
|  sushi|   12|
|chinese|   10|
+-------+-----+



In [ ]:
# import your tranformations as pandas df
# import pandas as pd
# pandas_df =dataframe.toPandas()
# use this wisely esp. with large datasets. can slow down 

In [ ]:
url ="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_16/wine.csv"
spark.sparkContext.addFile(url)
wine_df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True, encoding='UTF-8')

# Show DataFrame
wine_df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [ ]:
# Transformation: order points in descending order
wine_df.orderBy(wine_df["points"].desc())

DataFrame[country: string, description: string, designation: string, points: string, price: string, province: string, region_1: string, region_2: string, variety: string, winery: string]

In [ ]:
#Action:  show(5) method is an action that tells Spark to show the first five results.
wine_df.orderBy(wine_df["points"].desc()).show(5)

+-------+--------------------+--------------------+------+-----+----------+-----------+--------+--------------------+--------------------+
|country|         description|         designation|points|price|  province|   region_1|region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+-----------+--------+--------------------+--------------------+
|     US|This is an absolu...|           IX Estate|    99|  290|California|Napa Valley|    Napa|           Red Blend|              Colgin|
| France|98-100 Barrel sam...|       Barrel sample|    99| null|  Bordeaux|   Pauillac|    null|Bordeaux-style Re...|Ch̢teau Pontet-Canet|
|     US|There are incredi...|Elevation 1147 Es...|    99|  150|California|Napa Valley|    Napa|  Cabernet Sauvignon|        David Arthur|
| France|A magnificent Cha...|Dom P̩rignon Oeno...|    99|  385| Champagne|  Champagne|    null|     Champagne Blend|     Mo��t & Chandon|
|  Italy|Even better than .

In [ ]:
# aggregate functions
from pyspark.sql.functions import avg
wine_df.select(avg('points')).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [ ]:
# Filter df by condition
wine_df.filter("price<20").show(5)
# filtered using sql context

+--------+--------------------+-----------+------+-----+----------+--------------------+-----------------+--------------+--------------------+
| country|         description|designation|points|price|  province|            region_1|         region_2|       variety|              winery|
+--------+--------------------+-----------+------+-----+----------+--------------------+-----------------+--------------+--------------------+
|Bulgaria|This Bulgarian Ma...|    Bergul̩|    90|   15|  Bulgaria|                null|             null|        Mavrud|        Villa Melnik|
|   Spain|Earthy plum and c...|     Amandi|    90|   17|   Galicia|       Ribeira Sacra|             null|       Menc�_a|      Don Bernardino|
|      US|There's a lot to ...|       null|    90|   18|California|Russian River Valley|           Sonoma|    Chardonnay|            De Loach|
|      US|Massively fruity,...|       null|    91|   19|    Oregon|   Willamette Valley|Willamette Valley|    Pinot Gris|   Trinity Vineyards|

In [ ]:
# Filter by price on certain columns
# Using sql context
wine_df.filter("price<20").select('points','country','winery','price').show(5)

+------+--------+--------------------+-----+
|points| country|              winery|price|
+------+--------+--------------------+-----+
|    90|Bulgaria|        Villa Melnik|   15|
|    90|   Spain|      Don Bernardino|   17|
|    90|      US|            De Loach|   18|
|    91|      US|   Trinity Vineyards|   19|
|    91|Portugal|Adega Cooperativa...|   15|
+------+--------+--------------------+-----+
only showing top 5 rows



In [ ]:
# Same transformations with python

# Filter by price on certain columns
wine_df.filter("price<20").select(['points','country', 'winery','price']).show(5)

+------+--------+--------------------+-----+
|points| country|              winery|price|
+------+--------+--------------------+-----+
|    90|Bulgaria|        Villa Melnik|   15|
|    90|   Spain|      Don Bernardino|   17|
|    90|      US|            De Loach|   18|
|    91|      US|   Trinity Vineyards|   19|
|    91|Portugal|Adega Cooperativa...|   15|
+------+--------+--------------------+-----+
only showing top 5 rows



In [ ]:
wine_df.count()

150935

In [ ]:
# Filter on exact state 
wine_df.filter("country == 'US'").show(5)

+-------+--------------------+--------------------+------+-----+----------+------------------+-----------------+------------------+---------+
|country|         description|         designation|points|price|  province|          region_1|         region_2|           variety|   winery|
+-------+--------------------+--------------------+------+-----+----------+------------------+-----------------+------------------+---------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|       Napa Valley|             Napa|Cabernet Sauvignon|    Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|    Knights Valley|           Sonoma|   Sauvignon Blanc| Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|    Oregon| Willamette Valley|Willamette Valley|        Pinot Noir|    Ponzi|
|     US|This re-named vin...|              Silice|    95|   65|    Oregon|Chehalem Mountains|Willamette Valley|        Pinot Noir|Bergstr̦m|
|     

In [ ]:
#Using SQL context: filter to find the rows that contain a bottle of wine over $15 and that comes from California.
wine_df.filter("price > 15 and province == 'California'").show(5)

+-------+--------------------+--------------------+------+-----+----------+--------------------+--------+------------------+------------+
|country|         description|         designation|points|price|  province|            region_1|region_2|           variety|      winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+--------+------------------+------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|    Napa|Cabernet Sauvignon|       Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|  Sonoma|   Sauvignon Blanc|    Macauley|
|     US|The producer sour...|Gap's Crown Vineyard|    95|   60|California|        Sonoma Coast|  Sonoma|        Pinot Noir|   Blue Farm|
|     US|This blockbuster,...|     Rainin Vineyard|    95|  325|California|Diamond Mountain ...|    Napa|Cabernet Sauvignon|        Hall|
|     US|This fresh and li...|Gap'

In [ ]:
# Python context: 
wine_df.filter(wine_df['price']>200).show(5)


+-------+--------------------+-----------------+------+-----+----------------+--------------------+--------+------------------+------------------+
|country|         description|      designation|points|price|        province|            region_1|region_2|           variety|            winery|
+-------+--------------------+-----------------+------+-----+----------------+--------------------+--------+------------------+------------------+
|     US|This tremendous 1...|Martha's Vineyard|    96|  235|      California|         Napa Valley|    Napa|Cabernet Sauvignon|             Heitz|
|     US|This blockbuster,...|  Rainin Vineyard|    95|  325|      California|Diamond Mountain ...|    Napa|Cabernet Sauvignon|              Hall|
| France|Coming from a sev...|    Le Pigeonnier|    95|  290|Southwest France|              Cahors|    null|            Malbec|Ch̢teau Lagr̩zette|
|  Spain|Tarry blackberry ...|       Termanthia|    95|  220|  Northern Spain|                Toro|    null|     Tinta

In [ ]:
# Python context - filter to find the rows that contain a bottle of wine over $15 and that comes from California. 
wine_df.filter((wine_df.price > 15) & (wine_df.province == 'California')).show(5)

+-------+--------------------+--------------------+------+-----+----------+--------------------+--------+------------------+------------+
|country|         description|         designation|points|price|  province|            region_1|region_2|           variety|      winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+--------+------------------+------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|    Napa|Cabernet Sauvignon|       Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|  Sonoma|   Sauvignon Blanc|    Macauley|
|     US|The producer sour...|Gap's Crown Vineyard|    95|   60|California|        Sonoma Coast|  Sonoma|        Pinot Noir|   Blue Farm|
|     US|This blockbuster,...|     Rainin Vineyard|    95|  325|California|Diamond Mountain ...|    Napa|Cabernet Sauvignon|        Hall|
|     US|This fresh and li...|Gap'